# Step 3: Perform fit

In [ ]:
import numpy as np

import yaml

from tensorwaves.physics.helicityformalism.kinematics import HelicityKinematics
from tensorwaves.physics.particle import extract_particles
from tensorwaves.physics.helicityformalism.amplitude import IntensityBuilder

In [ ]:
with open("amplitude_model_helicity.yml") as input_file:
    recipe = yaml.load(input_file.read(), Loader=yaml.SafeLoader)
kinematics = HelicityKinematics.from_recipe(recipe)
particles = extract_particles(recipe)
data_sample = np.load("data_sample.npy")
phsp_sample = np.load("phsp_sample.npy")
intensity_builder = IntensityBuilder(particles, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity(recipe)
phsp_set = kinematics.convert(phsp_sample)
data_set = kinematics.convert(data_sample)

## 3.1 Define estimator

To perform a fit, you need to define an **estimator**. An estimator is a measure for the discrepancy between the intensity model and the data distribution to which you fit. In PWA, we usually use an *unbinned negative log likelihood estimator*, which can be created as follows with the `tensorwaves.estimator` module:

In [ ]:
from tensorwaves.estimator import UnbinnedNLL

estimator = UnbinnedNLL(intensity, data_set)

## 3.2 Modify parameters

In [ ]:
initial_parameters = estimator.parameters
initial_parameters

In [ ]:
initial_parameters["Width_f0(980)"] = 0.2
initial_parameters["Mass_f0(1500)"] = 1.6

## 3.3 Optimize the intensity model

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel('ERROR')

In [ ]:
from tensorwaves.optimizer.minuit import Minuit2

minuit2 = Minuit2()
result = minuit2.optimize(estimator, initial_parameters)
result

## 3.4 Export fit result

Optimizing an intensity model can take a long time, so it is important that you store the fit result in the end.